<center>
    <font size=5>Home and Cabin 
        Power Consumption 2021</font>
</center>

##### Sources:
Shivam Bansal - [Best of Kaggle Notebooks #2 - Time Series](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/discussion/276912)

# General
Notebook to analyse electricity consumption in my parents house and in the cottage.

<u><font size=4>Motivation / Objective:</font></u>
* Investigate the dataset using interactive plotting tools in python
* Catch trends seasonal and cyclic patterns in data
* Forecast power consumption based on time-series data

<u><font size=4>Data:</font></u>
* **Data type:** Tabular Data
    Hourly Power Consumption Dataset:
    * Data source: Eesti Energia AS, Estonian main electricity prowider company
    * Data download date: 25.01.2021
    * Data range: 01.01.2021 00:00 - 01.01.2022 00:00
    * Data given: hourly consumption rate in **kwh** - kilotwatt-hours

    Monthly Power Consumption Dataset:
    * Data source: Eesti Energia AS, 
    * Data download date: 25.01.2021
    * Data range 2020-2022
    * Monthly consumption summary statistics for years 2020 & 2021:
        * Daily 
        * Nightly
        * Total
* **Problem Type:** Predict Power consumption Supervised Time-Series Regresion

# Imports

In [348]:
import pandas as pd
import numpy as np
import re

from scipy import stats 
from scipy.stats import skew, norm, skewtest, linregress, pearsonr
from scipy.signal import periodogram

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf

import pandas_bokeh 

from bokeh.io import curdoc
from bokeh.layouts import column, row, gridplot
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Label, CDSView, GroupFilter, \
    HoverTool, RangeTool, BoxSelectTool, \
    Range1d, LinearAxis, LegendItem, Legend, \
    NumeralTickFormatter, DatetimeTickFormatter
from bokeh.palettes import Category10, Category20, Plasma, Plasma256, Viridis256

#from bokeh_plots import density_hist

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

In [349]:
pandas_bokeh.output_notebook()

# default plot theme for bokeh
curdoc().theme = 'dark_minimal'

Loading BokehJS ...

# Data

## Classes, functions

In [3]:
## CLASSES ##
class DFDtypeMapper(BaseEstimator, TransformerMixin):
    """Remap pandas dataframe dtypes.
    Parameters
    ----------
    dtype_dict : dict, {'dtype':[col_name]}
        Dictionary of dtypes as keys and values as list of column names. 
    
    Returns
    -------
    DataFrame : pd.DataFrame"""
    def __init__(self, dtype_dict : dict):
        self.dtype_dict = dtype_dict
        self.transformed_column_names = None 
    
    def fit(self, X, y=None):
        self.all_columns_ = X.columns
        return self
    
    def get_feature_names_out(self, input_features=None) -> np.ndarray:
        check_is_fitted(self)
        return self.all_columns_
    
    def transform(self, X, y=None) -> pd.DataFrame:
        X_ = X.copy()
        # remove columns that are not in X
        _dtype_dict = {}
        for dtype, val in self.dtype_dict.items():
            if isinstance(val, str):
                if val in X_.columns: 
                    _dtype_dict[dtype] = val
            elif type(val) not in [tuple, list, np.ndarray]:
                raise ValueError(f'Wrong type for {self.dtype_dict} value.')
            else:
                _dtype_dict[dtype] = [col for col in val 
                                           if col in X_.columns]
        
        for dtype in _dtype_dict:
            X_[_dtype_dict[dtype]] = X_[_dtype_dict[dtype]].astype(dtype)
        
        return X_

class DFValueMapper(BaseEstimator, TransformerMixin):
    """Rename values in column based on dictionary.
    Parameters
    ----------
    map_dict : dict 
        Dictionary of old mappings to new.
    cat_only : bool, default True
        - If True: consider category dtype columns only
        - If False: apply to all columns. Computationally more expensve.
    
    Returns
    -------
    DataFrame : pd.DataFrame
        Remapped pandas DataFrame."""
    def __init__(self, map_dict : dict, cat_only=True):
        self.cat_only = cat_only
        self.map_dict = map_dict
    def fit(self, X, y=None):
        self.all_columns_ = X.columns
        return self
    def get_feature_names_out(self, input_features=None) -> np.ndarray:
        check_is_fitted(self)
        return self.all_columns_
    def transform(self, X, y=None) -> pd.DataFrame:
        X_ = X.copy()
        # categorical features
        if self.cat_only:
            cat_cols = X_.columns[(X_.dtypes == 'category').values]
            X_[cat_cols] = X_[cat_cols].apply(
                lambda x: x.cat.rename_categories(self.map_dict))
            return X_
        else:
            return X_.replace(self.map_dict)

## FUNCTIONS ##
def datetime_gaps(df : pd.DataFrame, column : str, freq='D'):
    """Display time series frequencies and gaps.
    
    Parameters
    ----------
    column : str, DataFrame column or index name.
    freq : str, default 'D'
        Predominant frequency of the datetime column/index."""
    
    df = df.reset_index()
    date_range = pd.date_range(df[column][0], df[column].iloc[-1], freq=freq)
    df[column] = df[column].astype(f"period[{freq}]")
    
    # find frequencies
    temp = df.groupby([column]).sum().reset_index()
    freqs = (temp.loc[:,column]# frequencies
        .diff()
        .value_counts(dropna=False)
        .to_frame())
    print(f"Frequencies")
    display(freqs)
    
    # find gaps
    gaps = date_range.difference(df[column])
    if len(gaps) == 0:
        print(f"No gaps in {column}.")
    else:
        print(f"{len(gaps)} gaps in datetime:")
        return gaps



## Hourly Usage

In [4]:
# load the data
hourly = pd.read_csv(
    'data/tarbimine_tund.csv', 
    header=4, sep=';',
    index_col=False,
    names=['start', 'end', 'cabin', 'home'],
    parse_dates=['start', 'end'],
    decimal=',')

hourly.head(3)

,start,end,cabin,home
0,2021-01-01 00:00:00,2021-01-01 01:00:00,0.16,0.860
1,2021-01-01 01:00:00,2021-01-01 02:00:00,0.12,0.737
2,2021-01-01 02:00:00,2021-01-01 03:00:00,0.12,1.377


In [5]:
hourly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   start   8760 non-null   datetime64[ns]
 1   end     8760 non-null   datetime64[ns]
 2   cabin   8759 non-null   float64       
 3   home    8759 non-null   float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 273.9 KB


## Monthly Usage
Data in summarized tabular form:
* 3 location chunks:
    * cabin
    * home
    * both summarized
* 3 year chuncks per location
    * 2020 fully
    * 2021 fully
    * 2022 partially
* Each year with monthly index and overall summary in the end.

In [6]:
# read in the data with location headings
monthly_use = pd.read_csv(
    'data/tarbimine.csv', 
    header=None, sep=';',
    skiprows=4,
    index_col=False,
    decimal=',',
    names=['month', 'day', 'night', 'total'])

monthly_use.head()

,month,day,night,total
0,Mõõtepunkti aadress: Sutu,NaN,NaN,NaN
1,Tarbimine 2020. aastal,NaN,NaN,NaN
2,NaN,Päev (kWh),Öö (kWh),Kokku (kWh)
3,Jaanuar,"39,153","40,772","79,925"
4,Veebruar,"31,930","38,141","70,071"


### Clean & Reshape

In [7]:
temp = monthly_use.copy()

# references where to break the temp
locs = ['Sutu', 'Kuressaare', 'summeeritud']

# extract location
temp['locale'] = temp.month.str.extract(fr"({locs[0]}$|{locs[1]}$|{locs[2]})")
temp['locale'] = temp.locale.fillna(method='ffill')
temp['locale'] = temp.locale.rename({'Sutu':'cabin',
                                 'Kuressaare':'home',
                                 'summeeritud': 'total'})
# extract year
temp['year'] = temp.month.str.extract(r"^Tarbimine\s(\d{4})\.\saastal")
temp['year'] = temp.year.fillna(method='ffill')

# drop unneccessary rows
temp.dropna(inplace=True) # get rid of references

# convert comma decimals to points
temp.loc[:, 'day':'total'] = (
    temp.loc[:, 'day':'total']
    .apply(lambda x: x.str.replace(',', '.')))

# conversion dictionaries
dtype_dct = {'category':['month', 'locale', 'year'],
             'float':['day', 'night', 'total']}
map_dct = {  'Jaanuar':'Jan',
             'Veebruar': 'Feb',
             'Märts': 'Mar',
             'Aprill': 'Apr',
             'Mai': 'May',
             'Juuni': 'Jun',
             'Juuli': 'Jul',
             'August': 'Aug',
             'September': 'Sep',
             'Oktoober': 'Oct',
             'November': 'Nov',
             'Detsember': 'Dec',
             'Aasta kokku': 'Yearly',
             'Sutu': 'cabin',
             'Kuressaare': 'home',
             'summeeritud': 'total'}

# convert dtypes & remap values
temp = DFDtypeMapper(dtype_dct).fit_transform(temp)
temp = DFValueMapper(map_dct).fit_transform(temp)
monthly_stacked = temp.reset_index(drop=True)
monthly_stacked.head(3)

,month,day,night,total,locale,year
0,Jan,39.153,40.772,79.925,cabin,2020
1,Feb,31.930,38.141,70.071,cabin,2020
2,Mar,27.137,31.569,58.706,cabin,2020


In [8]:
# write cleaned df to csv
# monthly_stacked.to_csv('data/monthly_useage_clean.csv', index=False)

In [9]:
monthly = (
    monthly_stacked
    .query("year in ['2020','2021']")
    .set_index(['year', 'locale', 'month'])
    .sort_index())
monthly

day     night     total
year locale month                               
2020 home   Yearly  2598.340  2000.977  4599.317
            Apr      183.016   140.080   323.096
            Aug      207.633   164.666   372.299
            Dec      289.374   176.477   465.851
            Jan      211.679   162.732   374.411
...                      ...       ...       ...
2021 total  Mar      315.632   228.950   544.582
            Nov      310.882   187.765   498.647
            Oct      202.362   196.871   399.233
            Sep      200.281   163.210   363.491
            Feb      410.174   350.375   760.549

[78 rows x 3 columns]

## Holidays

In [10]:
holidays = pd.read_csv('data/holidays_estonia_2021.csv',
                       parse_dates=['date'])
holidays.head(3)

,date,description,type
0,2021-01-01,Uusaasta,"Riigipüha, puhkepäev"
1,2021-02-24,"Iseseisvuspäev, Eesti Vabariigi aastapäev","Rahvuspüha, puhkepäev"
2,2021-04-02,Suur reede,"Riigipüha, puhkepäev"


In [11]:
holidays.dtypes

date           datetime64[ns]
description            object
type                   object
dtype: object

# EDA

## Functions

In [263]:
def density_hist(df : pd.DataFrame, 
                 feature : str, 
                 bins : int=50, 
                 plot_height : int=400, 
                 plot_width : int=700, 
                 **fig_kwargs):
    '''Plots numpy histogram with probability density function value at the
    bin, normalized such that the integral over the range is 1. Additionally
    plots PDF and Cumulative Density Function on the same figure.
    
    Parameters
    ----------
    df : pd.DataFrame
        DF of the data.
    feature :  str
        Column name of the df.
    bins : int, default 50
        Number of equally spaced bins to plot.
        
    Returns
    -------
    fig : bokeh.plotting.figure.Figure
    '''
    # not nan feature values
    x = df[feature][df[feature].notna()].values 
    
    # Get the values for the histogram and bin edges (length(hist)+1)/
    # Use density to plot pdf and cdf on the same plot.
    hist, edges = np.histogram(x, bins=bins, density=True)
    
    ### PDF & CDF ##
    
    # find normal distribution parameters
    mu, sigma = norm.fit(x)
    xs = np.linspace(min(x), max(x)+1, len(x)) #x values to plot the line(s)
    
    pdf = norm.pdf(xs, loc=mu, scale=sigma) 
    cdf = norm.cdf(xs, loc=mu, scale=sigma) 
    
    # data sources for cdf
    source_cdf = ColumnDataSource(
        {'pdf':pdf, 'cdf':cdf, 'xs':xs, 'cdf_pc':cdf * 100})
    
    # create the canvas
    fig = figure(title=f"{feature} distribution", plot_height=plot_height,
                 plot_width=plot_width,x_axis_label=feature, 
                 y_axis_label='Density', **fig_kwargs)
    
    # add histogram
    fig.quad(bottom=0, top=hist, left=edges[:-1], right=edges[1:],
             fill_color='royalblue', line_color='black', alpha=0.7,
             legend_label=feature)

    # add pdf plot and hovertool 
    pdf_plot = fig.line('xs', 'pdf', source=source_cdf, line_color='red', 
                       line_width=5, alpha=0.5, legend_label='PDF')
    
    fig.add_tools(HoverTool(renderers=[pdf_plot], 
                           tooltips=[('PDF', '@pdf')],
                           mode='vline'))
    
    # set left-hand y-axis range
    fig.y_range = Range1d(0, max(hist) * 1.05)
    
    # setting the second y axis range name and range
    fig.extra_y_ranges = {"cdf": Range1d(start=0, end=1.05)}
    
    # adding the second y axis to the plot and to the right.  
    fig.add_layout(
        LinearAxis(y_range_name="cdf", axis_label='CDF',
                   formatter=NumeralTickFormatter(format="0%")), 'right')
    
    # add cdf with y range on the right and hovertool
    cdf_plot = fig.line('xs', 'cdf', source=source_cdf, alpha=0.8, 
                       line_color='darkgoldenrod', line_width=5, 
                       legend_label='CDF', y_range_name='cdf', name='cdf')
    
    fig.add_tools(HoverTool(renderers=[cdf_plot], 
                           tooltips=[('CDF', '@cdf_pc{0.0}%')],
                           mode='vline'))

    # figure properties
    fig.xgrid.visible = False
    
    # hide entries when clicking on a legend
    fig.legend.click_policy="hide"
    
    # add figure below for statistics
    fig_stats = figure(width=fig.width, height=50, outline_line_alpha=1,
                       toolbar_location=None, x_range=[0,1], 
                       y_range=[0,1], tools='')
    
    # set the components of the stats figure invisible
    for fig_component in [fig_stats.grid[0], fig_stats.ygrid[0],
                          fig_stats.xaxis[0], fig_stats.yaxis[0]]:
        fig_component.visible = False
    
    # statistics to test normality
    skew_test = skewtest(df[feature])
    skew_ = skew(df[feature])
    
    # annotation texts
    skewtest_label = Label(x=0.5, y=0.5, text_color='#FFFFFF', 
        render_mode='canvas', text_font_size='14px',
        text=f"Skewtest:", text_align='center')
    
    skew_zscore = Label(x=0.1, y=0.05, text_color='#FFFFFF', 
        render_mode='canvas', text_font_size='13px',
        text=f"z-score: {skew_test[0]:.2f}", text_align='left')
    
    skew_pvalue = Label(x=0.5, y=0.05, text_color='#FFFFFF', 
        render_mode='canvas', text_font_size='13px',
        text=f"p-value: {skew_test[1]:.2f}", text_align='center')
    
    skew_value = Label(x=0.9, y=0.05, text_color='#FFFFFF', 
        render_mode='canvas', text_font_size='13px',
        text=f"skew: {skew_:.2f}", text_align='right')
    
    # add annotations to the plot
    fig_stats.add_layout(skewtest_label)
    fig_stats.add_layout(skew_zscore)
    fig_stats.add_layout(skew_pvalue)
    fig_stats.add_layout(skew_value)
    
    return show(column([fig, fig_stats]))

def box(df: pd.DataFrame, 
        x: str, 
        y: str, 
        sort_by: str=None,
        ascending: bool=True,
        title: str=None,
        xlabel: str=None,
        ylabel: str=None,
        y_axis_format: str='0,0',
        hover_mean_format: str= '0.0',
        height: int=400,
        show_figure: bool=True,
        **fig_kwargs):
    """Plots bokeh box plot to show distributions with respect to categories.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with all necessary data.
    x : str
        DataFrame column name for the x axis.
    y : str
        DataFrame column name for the y axis.
    sort_by : str, default None
        If None the resulting boxes are sorted by mean value. Boxes are sorted
        by provided DataFrame column name. 
    ascending : bool, default True
        Wether to sort categories ascendingly or descendigly when sort == True.
    title : str, default 'Distribution of y by x'
        Title of the figure.
    x_label : str, default x
        X axis label.
    y_label : str, default y
        Y axis label.
    y_axis_format : str, default '0,0'
        Bokeh NumeralTickFormatter number format.
    hover_mean_format : str, default '0.0'
        Bokeh HoverTool tooltip format.
    height : int, default 400
        Figure height in pixels.
    show_figure : bool, default True
        Weather to show figure or return the bokeh.figure axis.   
    fig_kwargs : key-word arguments
        Key-word arguments for main figure. E.g. width, height, title.
    Returns
    -------
    fig : bokeh.figure
    """
    
    # string class values sorted alphabetically numeric ascendingly
    classes = sorted(df[x].unique())
    
    # group data by feature class values, returns DataFrameGroupBy object.
    groups = df[[x,y]].groupby(x)
    
    # find the quartiles and IQR for each class in the categorical data.
    # returned values are df-s
    q_25 = groups.quantile(q=0.25)
    q_50 = groups.quantile(q=0.5)
    q_75 = groups.quantile(q=0.75)
    iqr = q_75 - q_25
    upper = q_75 + 1.5*iqr
    lower = q_25 - 1.5*iqr

    # find the outliers for each class
    def outliers(group):
        class_name = group.name
        return group[(group[y] > upper.loc[class_name][y]) |
                     (group[y] < lower.loc[class_name][y])][y]
    
    # multindex (class, range index) series
    out = groups.apply(outliers).dropna().reset_index() 

    # sort outlier index values by mean class mean value
    out = pd.merge(left=q_50, right=out, how='outer', on=x, 
                   suffixes=('_mean', '')).sort_values(
                   by=y+'_mean').set_index(x)
    
    # construct outlier coordinates (if outliers excist)
    if not out.empty:
        outx = out.index.values.astype('str') # class names for x coordinates
        outy = out[y] # y values
    
    # if no outliers, shrink lengths of stems to be 
    # no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    lower[y] = [max([x2,y2]) for (x2,y2) in zip(list(qmin.loc[:,y]), lower[y])]
    upper[y] = [min([x2,y2]) for (x2,y2) in zip(list(qmax.loc[:,y]), upper[y])]

    # defining colors for n number of classes
    n = len(classes) 
    if n <= 1:
        colors = ['blue']
    if n == 2:
        colors = ['blue','green']
    if n <= 20:
        colors = Category20[n]
    else:
        # select colors uniformly along the spectrum to have variance
        every_nth = 256 // n
        colors = [Viridis256[i] for i in np.arange(0, every_nth * n, 10)]
    
    # create df of the data for the boxes
    box_df = pd.DataFrame(data={
        'classes':classes, 'q_25':q_25[y], 'q_50':q_50[y],'q_75':q_75[y],
        'upper':upper[y], 'lower':lower[y], 'color':colors})
    
    # add sort_by column to data
    if sort_by is not None:
        box_df[sort_by] = box_df['classes'].apply(lambda i: df[df[x]==i][sort_by][0])

    # force 'class' dtype to be str
    box_df['classes'] = box_df['classes'].astype(str)
    
    # sort data by class values (alphabetically or ascendingly)
    box_df = box_df.sort_values(by=sort_by, ascending=ascending) \
             if sort_by is not None else box_df.sort_values(by='q_50', ascending=ascending)
    
    # creating the bokeh source obj
    source = ColumnDataSource(box_df)
    
    # creating the figure
    title = f"Distribution of {y} by {x}" if title is None else title
    xlabel = x if xlabel is None else xlabel
    ylabel = y if ylabel is None else ylabel
    
    fig = figure(x_range=box_df.classes.unique(), title=title, 
                 x_axis_label=xlabel, y_axis_label=ylabel, height=height,
                 **fig_kwargs)
    
    # box color specs based on dark or light background
    line_color = curdoc().theme._json['attrs']['Axis']['major_tick_line_color']
    
    # stems
    fig.segment(x0='classes', y0='lower', x1='classes', y1='q_25', 
              line_color=line_color, source=source, name='seg')
    fig.segment(x0='classes', y0='upper', x1='classes', y1='q_75', 
              line_color=line_color, source=source, name='seg')
    
    # boxes
    boxes = fig.vbar(x='classes', bottom='q_25', top='q_75', width=0.7, 
                     line_color=line_color, color='color', source=source, 
                     name='box',hover_fill_color='maroon')
    
    # means
    means = fig.rect(x='classes', y='q_50', width=0.7, height=0.02, 
                     source=source, color=line_color, name='mean',
                     hover_fill_color='blue')
    
    # add hover to show the mean value
    fig.add_tools(HoverTool(renderers=[boxes], 
                            tooltips=[('Mean', f'@q_50{{{hover_mean_format}}}')], 
                            names=['box'], point_policy='snap_to_data'))
    
    # whiskers
    fig.rect(x='classes', y='lower', width=0.2, height=0.01, 
           line_color=line_color, source=source, name='rect')
    fig.rect(x='classes', y='upper', width=0.2, height=0.01, 
           line_color=line_color, source=source, name='rect')
    
    source2 = ColumnDataSource(data={'outx':outx, 'outy':outy})
    
    # outliers
    if not out.empty:
        plot_out = fig.circle(x='outx', y='outy', size=6, source=source2,
            color='grey', fill_alpha=0.3, name='outl', legend_label='outliers',
            hover_fill_color='maroon', hover_line_color='maroon')
        
        plot_out.visible = False
        fig.add_tools(HoverTool(renderers=[plot_out], tooltips=[(y,'@outy')],
                                names=['outl'], point_policy='snap_to_data'))
    
    # clickable legend
    fig.legend.click_policy = 'hide'
    fig.legend.background_fill_alpha=0.75
    
    # figure props
    fig.xgrid.visible = False
    fig.ygrid.visible = False
    fig.yaxis[0].formatter = NumeralTickFormatter(format=y_axis_format)
    
    # show or return fig
    if show_figure: show(fig)
    else: return fig

def calendar(
    df : pd.DataFrame,
    ys : [str],
    exclude_values_dict : dict={}, 
    include_values_dict : dict={},
    ylabel : str=None,
    hover_format : str=None,
    show_figure : bool=True,
    **fig_kwargs):
    """Plot (multi)line time series with other informative features. Bool dtypes
    are included by default.
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with all data to be plotted with datetime index dtype.
    ys : list, [str]
        List of column names to be plotted as time series y values. First 
        column name in the list is used for y_values for other columns.
    exclude_values_dict : dict, default {}
        Dictionary of the form {column:[value_1, ... ,value_n] to be excluded
        from plotting.
    include_values_dict : dict, default {}
        Dictionary of the form {column:[value_1, ... ,value_n] to be included
        to the resulting plot.
    ylabel : str, default None
        Y-label for time series y axis.
    hover_format : str, default None
        Format of the hover tool datetime values. 
    show_figure : bool, default True
        Weather to show figure or return the bokeh.figure axis.
    fig_kwargs : key-word arguments
        Key-word arguments for main figure. E.g. width, height, title.
        
    Returns
    -------
    fig : bokeh.figure"""
    
    # dictionaries to contain different column names
    len_exclude_dict = len(exclude_values_dict)
    len_include_dict = len(include_values_dict)
    if len_exclude_dict > 0 and len_include_dict > 0:
        if len(set(exclude_values_dict.keys()) & 
               set(include_values_dict.keys())) != 0:
            raise ValueError(f"Dictionaries can't contain same columns!")
    
    x = df.index.name                           # index name
    source = ColumnDataSource(df.reset_index()) # create bokeh source
    
    if ylabel == None: ylabel=ys[0]             # y-axis label
        
    # MAIN FIGURE #
    fig = figure(
        y_axis_label=ylabel,
        x_axis_type='datetime',
        **fig_kwargs)
    
    start_index = int(0.75 * len(source.data[x])) # explicitly set initial
    start = source.data[x][start_index]           # range for the figure
    end = source.data[x][-1]
    fig.x_range = Range1d(start, end)
    
    # RANGETOOL FIGURE #
    fig_rangetool = figure(
        title='Range Tool',
        height=130, 
        width=fig.width, 
        y_range=fig.y_range,
        x_axis_type='datetime',
        y_axis_type=None,
        tools="",
        toolbar_location=None,
    )
    
    range_tool = RangeTool(x_range=fig.x_range)
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.4

    fig_rangetool.ygrid.grid_line_color = None
    fig_rangetool.add_tools(range_tool)
    fig_rangetool.toolbar.active_multi = range_tool
    
    # x-axis tick format
    x_range_delta = df.index[-1] - df.index[0]  # data range in days
    dt_formatter = DatetimeTickFormatter(
        milliseconds=["%H:%M:%S.%f"],
        seconds=["%H:%M:%S"],
        minutes=["%H:%M:%S"],
        hours=["%H:%M:%S"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
    if x_range_delta <= pd.Timedelta(366, unit='D'):
        dt_formatter.days = ["%b"]
        dt_formatter.months = ["%b"]
        dt_formatter.years = ["%Y"]
        dt_hover_format = "%d %b"
        
    dt_hover_format = dt_hover_format if hover_format is None else hover_format 
    fig.xaxis.formatter = dt_formatter
    fig_rangetool.xaxis.formatter = dt_formatter
    
    # DATA #
    bools = df.select_dtypes(bool).columns
    features = [*ys, *bools, *exclude_values_dict.keys(), 
                *include_values_dict.keys()]
    n_features = len(features) # number of features
    # determine glyph colors
    palette = Category10 if n_features < 11 else Category20
    colors = ['blue', 'orange'] if n_features < 3 else palette[n_features]
    
    legend_items = []
    all_renderers = []
    for name, color in zip(features, colors):
        # init hovertool for each glyph on main
        hover = HoverTool( 
            tooltips=[(x, f"@{x}{{{dt_hover_format}}}")],
            formatters={f"@{x}":'datetime'},
            mode='vline')
        
        glyph_main = None
        glyph_range = None
        if name in ys:
            # draw glyphs on mian and on range tool
            glyph_main = fig.line(x=x, y=name, source=source, 
                                  color=color, line_width=2)
            glyph_range = fig_rangetool.line(x=x, y=name, source=source, 
                                             color=color)
            # add glyphs to renderers
            renderers = [glyph_main, glyph_range]
            all_renderers += renderers
            
            hover.tooltips.append((name, f"@{name}"))
            legend_items.append(LegendItem(
                label=f" {name}", renderers=renderers))
            
        else: 
            cds = None
            if name in bools: 
                # query True values only 
                true_idx = df[name][df[name] == True].index
                temp_df = df[ys[0]].loc[true_idx].reset_index()
                cds = ColumnDataSource(temp_df)
            
            elif name in exclude_values_dict.keys(): # values to exclude
                idx = df[name][~df[name].isin(exclude_values_dict[name])] \
                    .index
                cds = ColumnDataSource(df.loc[idx,[ys[0],name]].reset_index())
            
            else: # values to include
                idx = df[name][df[name].isin(include_values_dict[name])] \
                    .index
                cds = ColumnDataSource(df.loc[idx,[ys[0],name]].reset_index())
            
            # create glyphs
            glyph_main = fig.circle(x=x, y=ys[0], source=cds, 
                color=color, size=8, alpha=0.5)
            glyph_range = fig_rangetool.circle(x=x, y=ys[0], 
                source=cds, color=color, size=4, alpha=0.5)
            
            # add glyps to rednerers list
            renderers = [glyph_main, glyph_range]
            all_renderers += renderers
            
            # hover and legend
            hover_string = "True" if name in bools else f"@{name}"
            hover.tooltips.append((name, hover_string))
            legend_items.append(LegendItem(
                label=f" {name}", 
                renderers=renderers))
        
        hover.renderers = [glyph_main] # for HoverTool
        all_renderers += renderers # for legend
        
        fig.add_tools(hover)

    # Dummy fig for legend
    fig_legend = figure(width=130, height=fig.height + 130, 
                        outline_line_alpha=0,toolbar_location=None,
                        border_fill_color='#ffffff')
    
    # set the components of the figure invisible
    for fig_component in [fig_legend.grid[0], fig_legend.ygrid[0],
                          fig_legend.xaxis[0], fig_legend.yaxis[0]]:
        fig_component.visible = False
    
    # set the figure range outside of the range of all glyphs
    fig_legend.renderers += all_renderers
    fig_legend.x_range.end = fig.x_range.end + pd.Timedelta(365, unit='D')
    fig_legend.x_range.start = fig.x_range.start + pd.Timedelta(360, unit='D')
    fig_legend.add_layout(Legend(click_policy = "hide", location='center', 
                                 items=legend_items, border_line_width=2))
    
    if show_figure:
        return show(row(column(fig,fig_rangetool), fig_legend))
    else:
        return row(column(fig,fig_rangetool), fig_legend)

def plot_periodogram(ts: [pd.Series, pd.DataFrame],
                 show_figure: bool=True,
                 **fig_kwargs):
    """Plot bokeh periodogram wrapped around scipy.signal.periodogogram.
    
    Parameters
    ----------
    ts : pd.Series or pd.Dataframe
        Pandas series or DataFrame with all columns to be plotted.
    show_figure : bool, default True
        Weather to show figure or return the bokeh.figure axis.
    fig_kwargs : key-word args
        Key-word arguments for plot_bokeh figure.
    Returns
    -------
    None
    """
    # periodogram nice xticks
    period_coefs = [1, 2, 4, 6, 12, 26, 52, 104] 
    periods = ["Annual (1)",
               "Semiannual (2)",
               "Quarterly (4)",
               "Bimonthly (6)",
               "Monthly (12)",
               "Biweekly (26)",
               "Weekly (52)",
               "Semiweekly (104)"]
    periods_dct = {coef:period for coef,period in zip(period_coefs, periods)}
    
    # convert ts to dataframe if series
    df = ts.to_frame() if isinstance(ts, pd.Series) else ts
    
    # determine sampling frequency
    sampling_f = pd.Timedelta('365D') / pd.Timedelta('1D')
    
    temp_df = pd.DataFrame() # init empty df
    for column in df.columns:
        # create periodogram profile(s)
        frequencies, spectrum = periodogram(
            x=df[column],
            fs=sampling_f,
            window='boxcar',
            detrend='linear',
            scaling='spectrum')
    
        if temp_df.empty: 
            temp_df.index = frequencies # add x if not already
        temp_df[column] = spectrum # add spectrum per column
    
    temp_df.index.name = 'period' # name the index
    
    # create period categorical names
    temp_df['period_cat'] = "Semiweekly"
    for i, period in enumerate(period_coefs):
        if i == len(period_coefs) - 1: 
            break
        start = 0 if i == 0 else (period_coefs[i-1] + period) / 2
        next_period = period_coefs[i+1]
        end = (next_period + period_coefs[i+2]) / 2 if i != 6 else 78
        temp_df.loc[start:end, 'period_cat'] = periods_dct[period][:-4]
    
    source = ColumnDataSource(temp_df.reset_index()) # init source
    
    # define pallette
    n_cols = len(temp_df.columns)
    colors = ['purple', 'goldenrod']
    pallette = Plasma[n_cols] if n_cols > 2 else \
               (colors[0] if n_cols == 1 else colors)
    
    # create the figure
    fig = figure(title="Periodogram",
                 x_axis_label="Period",
                 y_axis_label='Variance',
                 x_axis_type="log",
                 **fig_kwargs)
    
    # remap to nice xticks and rotate labels to fit
    fig.xaxis.ticker = period_coefs
    fig.xaxis.major_label_overrides = periods_dct
    fig.xaxis.major_label_orientation = np.pi / 6
    
    # create graphs and store renderers and legend items
    all_renderers = []
    legend_items = []
    for name, color in zip(temp_df.columns, colors):
        # add step glyphs
        step = fig.step(x='period', y=name, source=source, color=color, 
                        line_width=4, mode="before")
        
        circle = fig.circle(x='period', y=name, source=source, size=10,
                            alpha=0)
        renderers = [step, circle]
        hover = HoverTool(tooltips=[(name ,f"@{name}"), ("period", "@period_cat")], 
                          renderers=[circle], point_policy='snap_to_data', 
                          mode='vline')
        fig.add_tools(hover)
        
        legend_items.append(LegendItem(label=f"{name}", renderers=renderers))
        all_renderers += renderers
    
    # add legend
    fig.add_layout(Legend(click_policy='hide', items=legend_items))
    
    if show_figure:
        show(fig)
    else:
        return fig

def lag_plot(x: pd.Series, 
             y=None,
             lag: int=1, 
             lead: int=None, 
             standardize: bool=False,
             show_figure: bool=False,
             **fig_kwargs):
    """Plot n-th lag or lead scatter plot with regression fit.
    
    Parameters
    ----------
    x : pd.Series
        Pandas series of a feature to be plotted.
    y : array-like, default None
        Pre-computed lag or lead array of values.
    lag: int, default 1
        Shift x values n-units forward.
    lead : int, default None
        Shift x values n-units backward.
    standardize : bool, default False
        Wether to normalize the data or not.
    show_figure: bool, default False
        Return figure axis or show the figure.
    fig_kwargs : key-word args
        Figure key-word arguments like: plot_height, plot_width,
    
    Returns
    -------
    Figure : figure, None, default None
        If show_figure is True returns None (shwos the figure), otherwise
        returns figure to which axis is connected."""
    
    # check both lag and lead are not None
    if lag == lead == None:
        raise ValueError(f"Both lag and lead can't be None!")
    elif type(lag) == type(lead) == int:
        raise ValueError(f"Both lag and lead can't be specified!")
    
    # create lag or lead
    type_ = 'lag'
    number_ = lag
    if lead is not None:
        x_ = x.shift(-lead)
        type_ = 'lead'
        number_ = lead
    else:
        x_ = x.shift(lag)
    
    # standardize if specified
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    
    x_ = x_.dropna() # drop NaN-s
    x_, y_ = x_.align(y_, join='left')
    
    # init main figure
    fig = figure(
        title=f"{type_.capitalize()} {number_}", 
        y_axis_label=f"{x.name}", 
        x_axis_label=f"{x.name}_{type_}_{number_}",
        **fig_kwargs)
    
    source = ColumnDataSource({f"{x.name}{type_}":x_, x.name:y_})
    
    # plot scatter
    scatter = fig.circle(x=f"{x.name}{type_}", y=x.name, source=source, 
                         size=7, hover_fill_color='goldenrod',
                         hover_line_color='goldenrod')
    
    # plot regression fit
    slope, intercept, rvalue, pvalue, stderr = linregress(x=x_, y=y_)
    
    fig.line(x=x_, y=slope * x_ + intercept, color='red', alpha=0.75,
             line_width=5, legend_label=f"r = {rvalue:.2f}")
    
    # add hovertool
    hover = HoverTool(
        renderers=[scatter],
        tooltips=[(f"{x.name}", f"@{x.name}"),
                  (f"{type_}", f"@{x.name}{type_}")])
    
    fig.add_tools(hover)
    
    # hide entries when clicking on a legend
    fig.legend.click_policy="hide"
    
    if show_figure:
        return show(fig)
    else:
        return fig    

def plot_lags(x: pd.Series, 
              y=None,
              lags: [int, [int]]=None,
              leads: [int, [int]]=None,
              ncols: int=4,
              **grid_kwargs):

    
    # list of leads and lags actual values 
    leads_ = [*range(1,leads+1)] if type(leads) == int else leads
    lags_ = [*range(1,lags+1)] if type(lags) == int else lags
    
    all_plots = []
    if leads is not None:
        for lead in leads_:
            temp_fig = lag_plot(x=x, y=y, lag=None, lead=lead)
            all_plots.append(temp_fig)
    if lags is not None:
        for lag in lags_:
            temp_fig = lag_plot(x=x, y=y, lag=lag, lead=None)
            all_plots.append(temp_fig)
    
    grid = gridplot(all_plots, ncols=ncols, **grid_kwargs)
    show(grid)    
    
def correlogram(x: [[], np.array, pd.Series, pd.DataFrame], 
                lags: int, 
                zero: bool=False, 
                show_figure: bool=True, 
                **fig_kwargs):
    
    """Plot n specified autocorrelation lags. Based on 
    statsmodels.tsa.stattools.pacf estimate.
    
    Parameters
    ----------
    x : array-like
        Observations of time series for which pacf is calculated.
    lags : int
        Number of lags to return autocorrelation for. 
    zero : bool, default False
        Flag indicating whether to include the 0-lag autocorrelation.
    show_figure : bool, default True
        Weather to show figure or return the bokeh.figure axis.
    kwargs** : keyword arguments, optional
        Optional keyword arguments of bokeh.plotting.figure.Figure. E.g. 
        'plot_width', 'plot_height'. 
        
    Returns
    -------
    Figure : figure, None, default None
        If show_figure is True returns None (shwos the figure), otherwise
        returns figure to which axis is connected."""
    
    # lags as x-values
    xs = [*np.arange(0, lags+1)] if zero else [*np.arange(1, lags+1)]
    
    # partial autocorrelation values as ys and confidence intervals
    auto_corrs, ci = pacf(x=x, nlags=lags, alpha=.05) 
    
    # remove first 0th lag
    if not zero:
        auto_corrs = auto_corrs[1:]
        ci = ci[1:]
    
    ci = [arr - pa for arr, pa in zip(ci, auto_corrs)] # confidence intervals
    ci_cutoff = abs(ci[0][0])
    
    # prepare the data into df
    df = pd.DataFrame({'x': xs, 'y': auto_corrs})
    df['high_low'] = df['y'].apply(lambda x: 'high' if abs(x) >= ci_cutoff else 'low')
    df['xs_multi'] = df['x'].apply(lambda x: [x, x])
    df['ys_multi'] = df['y'].apply(lambda x: [0, x])
    
    # create CDS and Views
    source = ColumnDataSource(df)
    view_low = CDSView(source=source, 
        filters=[GroupFilter(column_name='high_low', group='low')])
    view_high = CDSView(source=source, 
        filters=[GroupFilter(column_name='high_low', group='high')])
    
    # init main figure
    fig = figure(**fig_kwargs, title='Partial Autocorrelation', 
                 x_axis_label=f'{x.name} Lags', y_axis_label='Partial Correlation')
    
    fig.x_range = Range1d(0, lags + 1) # explicitly set initial range
    baseline = fig.line(x=[-10, lags + 10], y=[0, 0], line_width=3)
    
    legend_items = []
    hover_renderers = []
    all_renderers = []
    for view_, type_, name_, size_ in zip(
        [view_low, view_high], ['circle', 'star'], 
        ['Insignificant','Significant'], [5, 9]):
        
        # draw vertical lines and circles
        multi_line = fig.multi_line(
            xs='xs_multi',
            ys='ys_multi',
            source=source,
            view=view_,
            line_color='#1f77b4',
            line_width=3,
            hover_line_color='goldenrod')
    
        # draw circles or stars
        glyphs = fig.scatter(x='x', y='y', source=source, view=view_, 
                             size=size_, color='#1f77b4', marker=type_,
                             hover_fill_color='goldenrod', 
                             hover_line_color='goldenrod')
        
        # store renderers
        renderers = [multi_line, glyphs]
        legend_items.append(LegendItem(label=f"{name_}", renderers=renderers))
        hover_renderers += [multi_line]
        all_renderers += renderers
        
    # add HoverTool    
    hover = HoverTool(renderers=hover_renderers,
                          tooltips=[('PACF', "@y{0.00}"),('Lag', "@x")],
                          line_policy='interp')
    fig.add_tools(hover)
        
    # draw 95% confidence intervals
    ci = fig.varea(x=xs, 
                   y1=[i[0] for i in ci],
                   y2=[i[1] for i in ci],
                   alpha=0.10)
    
    legend_items.append(LegendItem(label="95% CI", renderers=[ci]))
    fig.add_layout(Legend(click_policy='hide', items=legend_items))
    
    if show_figure:
        show(fig)
    else:
        return fig
    
def regplot(df: pd.DataFrame, 
            x: str, 
            y: str,
            reg: bool=True,
            alpha: float=0.95,
            extra_hover_tooltips: [tuple]=None,
            hover_formatters: dict={},
            show_figure=True,
            **fig_kwargs):
    '''Plots bokeh scatter plot.
    Parameters
    ----------
    df : dict / DataFrame
         Dictionary or pandas DataFrame where plotting data resides.
    x : str
         Key or column name for data in x-axis.
    y : str
         Key or column name for data in y-axis.
    reg : bool, default True
        To plot regression line or not. Default False.
    alpha : float, default 0.95
        Probability that random variable will be drawn from the returned 
        range. Confidence interval.
    extra_hover_tooltips : list of tuples, default None
        List of hover tool tooltips of the form [('name', '@name{}'), ... ,]
    hover_formatters : dict, default None
        Dictionary of hover tool tooltip formatters of the form 
        {'name': 'datetime'}.
        
    Returns
    -------
    Figure : bokeh.figure, default None
        IIf show_figure is True returns None (shwos the figure), otherwise
        returns figure to which axis is connected.'''
    
    df = df.reset_index()
    source = ColumnDataSource(df) # bokeh source object
    
    # init the figure/canvas for the plot
    fig = figure(**fig_kwargs, 
                 title=f"{x} vs {y}",
                 x_axis_label=x, 
                 y_axis_label=y)
    
    # add data to figure
    circle = fig.circle(x, y, size=10, alpha=0.75, source=source, 
                        hover_fill_color='goldenrod',
                        hover_line_color='goldenrod',
                        selection_fill_color='goldenrod')
    
    # synthetic x values for regression lines
    xs = np.linspace(df[x].min(), df[x].max() + 1, num=len(df[x]))
    colors = ['red', 'green']
    orders = ['1st Order', '2nd Order']
    legend_items = []
    
    # add regression lines and confidence intervals
    if reg: 
        for order, name, color in zip([1,2], orders, colors):
            # calculate Confidence Interval
            ci_95 = norm.interval(
                alpha=alpha,            # confidence interval
                loc=0,                  # mean of the distribution
                scale=stats.sem(df[y])) # standard error of the mean

            # find polynomial coefficients, highest degree to lowest
            coefs = np.polyfit(x=df[x], y=df[y], deg=order)
            ys = coefs[0] * xs + coefs[1] if order == 1 else \
                 (coefs[0] * xs ** 2) + (coefs[1] * xs) + coefs[2]

            # plot regression line and confidence intervals
            reg_line = fig.line(x=xs, y=ys, 
                                color=color, alpha=0.75, line_width=5)

            # plot regression confidnece intervals
            ci = fig.varea(x=xs, y1=ys + ci_95[0], y2=ys + ci_95[1],
                           alpha=0.25, color=color)
            
            # calculate Pearson correlation coefficient
            r = pearsonr(df[x] ** order, df[y])[0]
            
            # add legend items
            legend_items.append(LegendItem(label=f"{name} r = {r:.2f}", 
                                           renderers=[reg_line, ci]))
    # add hover tool
    hover = HoverTool(
        renderers=[circle], 
        tooltips=[(y, f"@{y}"), (x, f"@{x}")],
        formatters=hover_formatters)
    
    if extra_hover_tooltips is not None:
        hover.tooltips.append(*extra_hover_tooltips)
    
    # add tools and legend to figure
    fig.add_tools(hover, BoxSelectTool())
    fig.add_layout(Legend(click_policy='hide', items=legend_items))
    
    if show_figure:
        return show(fig)
    else:
        return fig



## Validate
##### Summary Stats
Validate summary statistics in <code>monthly</code> df.

In [13]:
hourly[['cabin', 'home']].sum()

cabin    1382.036
home     4663.473
dtype: float64

In [14]:
monthly.loc['2021',['cabin', 'home'],'Yearly'].total

year  locale  month 
2021  cabin   Yearly    1382.036
      home    Yearly    4663.473
Name: total, dtype: float64

## NaN-s

In [15]:
hourly_eda = hourly.copy()
hourly_eda[hourly_eda.isna().any(axis='columns')]

,start,end,cabin,home
2067,2021-03-28 03:00:00,2021-03-28 04:00:00,NaN,NaN


That NaN correspond to the switch from wintertime to daylight saving time in Estonia.

In [16]:
temp = hourly.copy()
temp = temp.set_index('end')[['cabin', 'home']]

# check duplicates
print(f"Has duplicates: {temp.index.has_duplicates}")

# potential hours missing in the data
print(f"n_rows: {temp.shape[0]}")
temp.index = temp.index.to_period(freq='H')
temp = temp.reset_index().groupby(['end']).cabin.sum().reset_index()
temp.end.diff().value_counts(dropna=False)

Has duplicates: False
n_rows: 8760


<Hour>    8759
NaT          1
Name: end, dtype: int64

No hours missing from the data. Check time series around when switching to wintertime at 4:00, last sunday in October. The the clock is turned back an hour.

In [17]:
temp = hourly.copy().set_index('start')
temp = temp[(temp.index.month==10) & # october
            (temp.index.weekday==6) & # sunday
            ((temp.index.hour>0) & (temp.index.hour<8))] # between 1am-8am
temp[temp.index.day == temp.index.day.max()].reset_index()

,start,end,cabin,home
0,2021-10-31 01:00:00,2021-10-31 02:00:00,0.607,0.171
1,2021-10-31 02:00:00,2021-10-31 03:00:00,0.095,0.138
2,2021-10-31 03:00:00,2021-10-31 04:00:00,0.138,0.958
3,2021-10-31 04:00:00,2021-10-31 05:00:00,0.153,0.174
4,2021-10-31 05:00:00,2021-10-31 06:00:00,0.062,0.137
5,2021-10-31 06:00:00,2021-10-31 07:00:00,0.056,0.153
6,2021-10-31 07:00:00,2021-10-31 08:00:00,0.106,0.864


Since there are no duplicated entries in the index we can assume that from 4am-5am holds summed data for 1 hour of sumemrtime and 1 hour of wintertime.

## Feature Engineering #1
In order to inspect the Time Series data we'are going to add some basic time-related information.

In [18]:
hourly_eda = hourly.copy()
hourly_eda = (
    hourly_eda
    .fillna(0)
    .rename({'end':'time'}, axis='columns')
    .set_index('time') # time as index
    .loc[:, 'cabin': 'home']) # drop start time

# add features
hourly_eda['month'] = hourly_eda.index.month
hourly_eda['day'] = hourly_eda.index.day
hourly_eda['hour'] = hourly_eda.index.hour
hourly_eda['day_of_week'] = hourly_eda.index.weekday
hourly_eda['is_weekend'] = hourly_eda.day_of_week > 4
hourly_eda['is_winter'] = (hourly_eda.month > 11) | (hourly_eda.month < 4)
hourly_eda['is_summer'] = (hourly_eda.month > 5) | (hourly_eda.month < 9)
hourly_eda.head(1)

,cabin,home,month,day,hour,day_of_week,is_weekend,is_winter,is_summer
time,,,,,,,,,
2021-01-01 01:00:00,0.16,0.86,1,1,1,4,False,True,True


##### Summer/Winter Time
* Transition to summertime (DST) : Last Sunday in March at 3:00
* Transition to wintertime : Last Sunday in October at 4:00

In [19]:
# last sunday in march at 3am
to_summer_time = (
    hourly_eda
    .query("month == 3 & day_of_week == 6 & hour == 4")
    .index.max())

# last sunday in october at 4am
to_winter_time = (
    hourly_eda
    .query("month == 10 & day_of_week == 6 & hour == 4")
    .index.max())

hourly_eda['is_dst'] = False
hourly_eda.loc[to_summer_time:to_winter_time, 'is_dst'] = True
hourly_eda.head(1)

,cabin,home,month,day,hour,day_of_week,is_weekend,is_winter,is_summer,is_dst
time,,,,,,,,,,
2021-01-01 01:00:00,0.16,0.86,1,1,1,4,False,True,True,False


##### Day & Night Rate
**Day rate:** 
* 7-23 during wintertime
* 8-24 during daylight saving time (DST)

**Night rate:**
* 23-7 during wintertime
* 24-8 during summertime
* during national holidays if it does not land on weekday

In [20]:
temp = hourly_eda.copy()

temp['rate'] = 'day'
temp.loc[
    temp.query("(hour <= 7 | hour > 23) & (is_dst == False)").index,
    'rate'] = 'night'
temp.loc[temp.query("hour <= 8 & is_dst == True").index, 'rate'] = 'night'
temp.loc[temp.query("is_weekend == True").index, 'rate'] = 'night'

hourly_eda = temp
hourly_eda.head(1)

,cabin,home,month,day,hour,day_of_week,is_weekend,is_winter,is_summer,is_dst,rate
time,,,,,,,,,,,
2021-01-01 01:00:00,0.16,0.86,1,1,1,4,False,True,True,False,night


##### Holidays

In [21]:
holidays.head()

,date,description,type
0,2021-01-01,Uusaasta,"Riigipüha, puhkepäev"
1,2021-02-24,"Iseseisvuspäev, Eesti Vabariigi aastapäev","Rahvuspüha, puhkepäev"
2,2021-04-02,Suur reede,"Riigipüha, puhkepäev"
3,2021-04-04,Ülestõusmispühade 1. püha,"Riigipüha, puhkepäev"
4,2021-05-01,Kevadpüha,"Riigipüha, puhkepäev"


In [22]:
temp = hourly_eda.copy()
hol = holidays.copy()
hol = hol.set_index('date')

temp['dummy_index'] = pd.to_datetime(temp.index.date)
temp = temp.reset_index().set_index('dummy_index')

# merge holidays to hourly
temp = temp.merge(hol['description'], how='left', 
                  left_index=True, right_index=True)

temp['is_holiday'] = temp.description.notna()
temp['description'] = temp.description.fillna('normal')

hourly_eda = temp.set_index('time')
hourly_eda.head(1)

,cabin,home,month,day,hour,day_of_week,is_weekend,is_winter,is_summer,is_dst,rate,description,is_holiday
time,,,,,,,,,,,,,
2021-01-01 01:00:00,0.16,0.86,1,1,1,4,False,True,True,False,night,Uusaasta,True


## Consumption

Inspect raw hourly consumption.

In [23]:
df_ts_h = hourly_eda.copy()[['home', 'cabin']]

a = calendar(df_ts_h,
             ys=['home', 'cabin'],
             ylabel='Consumption [kWh]',
             title='Hourly Power Consumption',
             hover_format="%H:00 %d %b",
             width=550, height=250)

The plotted data is too raw to inspect nicely, lets plot 12h moving averages instead.

In [24]:
df_ts_12h_mean = df_ts_h.rolling(window='12h').mean()

a = calendar(df_ts_12h_mean,
             ys=['home', 'cabin'],
             ylabel='Consumption [kWh]',
             title='12-Hour Mean Power Consumption',
             hover_format="%H:00 %d %b",
             width=550, height=350)

### Distributions
Examine hourly and daily power consumption.
##### Hourly

In [25]:
a = temp.plot_bokeh(
    y=['cabin', 'home'],
    kind='hist', 
    bins=40,
    histogram_type='sidebyside',
    hovertool=True,
    title="Hourly Power Consumption Distributions",
    line_color='white',
    line_width=1,
    ylabel='Counts',
    use_index=False,
    xticks=np.arange(0, 5.5, 0.5),
    xlabel='Consumption [kWh]',
    colormap=['blue', 'green', 'red'])

In [26]:
cabin_density = density_hist(temp, 'cabin', 40)
home_density = density_hist(temp, 'home', 40)

grid = pandas_bokeh.plot_grid([[cabin_density],[home_density]], 
                              width=700, height=300)

##### Daily

In [27]:
# daily energy consumption distributions
temp_1D = temp.resample("1D").sum()[['cabin', 'home']]

a = temp_1D.plot_bokeh(
    kind='hist', 
    bins=40,
    histogram_type='sidebyside',
    hovertool=True,
    title="Daily Power Consumption Distributions",
    line_color='white',
    line_width=1,
    ylabel='Counts',
    use_index=False,
    xticks=np.arange(0, 36, 5),
    xlabel='Consumption [kWh]',
    colormap=['blue', 'green', 'red'],
    number_format="1.000")

In [28]:
cabin_density = density_hist(temp_1D, 'cabin', 40)
home_density = density_hist(temp_1D, 'home', 40)

grid = pandas_bokeh.plot_grid([[cabin_density],[home_density]], 
                              width=700, height=300)

Both hourly and daily power consmption graphs are skewed. We will transform them to have more gaussian like shape when preparing the data for Machine Learning process.

### Seasonalities
Inspect time related consumption frequencies and seasonality.

#### Hour of Day

In [29]:
season_h = hourly_eda.copy()[['cabin', 'home']]
season_h.index.name = 'date_time' # rename index
season_h['hour'] = season_h.index.hour # extract time from index
season_h['hour'] = season_h.hour.replace({0:24})
season_h.head(2)

,cabin,home,hour
date_time,,,
2021-01-01 01:00:00,0.16,0.860,1
2021-01-01 02:00:00,0.12,0.737,2


In [155]:
# average & median consumption by hour
season_h_grouped = season_h.groupby(['hour']).mean().reset_index()
season_h_grouped_med = season_h.groupby(['hour']).median().reset_index()

h = season_h_grouped.plot_bokeh(
    kind='line',
    x='hour',
    y=['cabin', 'home'],
    title='Average Consumption by Time of Day',
    ylabel='Consumption',
    xlabel='Time of Day'
)

In [147]:
h = season_h_grouped_med.plot_bokeh(
    kind='line',
    x='hour',
    y=['cabin', 'home'],
    title='Median Consumption by Time of Day',
    ylabel='Consumption',
    xlabel='Time of Day'
)

In [158]:
season_h_avg_med = season_h_grouped[['home', 'cabin']].merge(
    season_h_grouped_med[['home', 'cabin']], how='left', left_index=True,
    right_index=True, suffixes=("_avg", "_med")
)
season_h_avg_med['hour'] = season_h_avg_med.index + 1
season_h_avg_med.head(3)

,home_avg,cabin_avg,home_med,cabin_med,hour
0,0.429170,0.152096,0.233,0.092,1
1,0.267860,0.131712,0.154,0.083,2
2,0.260389,0.142345,0.141,0.098,3


In [160]:
h = season_h_avg_med.plot_bokeh(
    kind='line',
    x='hour',
    y=['home_avg', 'home_med'],
    title='Average & Median Consumption by Time of Day',
    ylabel='Home Consumption',
    xlabel='Time of Day'
)

In [143]:
a = box(season_h, x='hour', y='home', sort=True, width=700, 
        ylabel="Home Consumption", xlabel="Time of Day")

In [144]:
a = box(season_h, x='hour', y='cabin', sort=True, width=700,
        ylabel="Home Consumption", xlabel="Time of Day")

**home** - has clear hourly seasonality, with morning and late afternoons incresaing consumption and afternoons and late evenings decreasing consumption profile.

**cabin** - doesn't have clear hourly seasonality profile

#### Day of Week

In [31]:
season_d = hourly_eda.copy()[['cabin', 'home']]
season_d.index.name = 'date_time' # rename index
season_d = season_d.resample('1D').sum() # resample from hourly to daily freq

# create day_of week feature
season_d['day_of_week'] = season_d.index.dayofweek
season_d.head(2)

,cabin,home,day_of_week
date_time,,,
2021-01-01,3.667,25.949,4
2021-01-02,3.741,10.719,5


In [162]:
season_d_grouped = season_d.groupby('day_of_week').mean().reset_index()

day_num_dict = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
season_d_grouped['day'] = season_d_grouped.day_of_week.replace(day_num_dict)

h = season_d_grouped.plot_bokeh(
    kind='line',
    x='day',
    y=['cabin', 'home'],
    title='Average Consumption by Day of Week',
    ylabel='Consumption',
    xlabel='Day of Week'
)

In [33]:
# find variation in %
season_d_home = season_d_grouped['home']
pct_diff = ((season_d_home.max() - season_d_home.min()) / season_d_home.max())
print(f"Max difference {pct_diff * 100:.2f}%")

Max difference 5.15%


There are not much variation on daily power consumption. Only marginal 5% between lowest (Tuesday) and highest (Thursday).

In [227]:
season_d_temp = season_d.copy()
season_d_temp['day'] = season_d_temp.day_of_week.replace(day_num_dict)

a = box(season_d_temp, x='day', y='home', sort_by='day_of_week',
        title='Distribution of Consumption by Day of Week',
        ylabel='Home Consumption', xlabel='Day')
a

In [279]:
season_d_grouped = season_d_temp.groupby('day').mean().reset_index()

a = season_d_grouped.plot_bokeh.pie(
    x='day', y=['cabin', 'home'],
    colormap=Category20[7],
    title="Power Consumption by Day of Week",
    figsize=(500,500),
    xlabel='Day',
    ylabel='Consumption',
    hovertool=True,
    line_color='white'
)

__x__values_original


#### Periodogram
Let's investgate longer term seasonality from the periodogram plot.

In [34]:
plot_periodogram(season_d[['home', 'cabin']], width=700, height=400)

**home** & **cabin** : we can distindguish 2 major peaks in the poweer spectral profile, <code>Annual</code> and <code>Monthly</code>. 2 peaks = 4 Fourier features later on in the time analysis. There is a peak between Biweekly and Weekly variance but weekly variations we are going to capture with indicators - one hot encode day_of_week.

#### Monthly

In [36]:
season_m = hourly_eda.copy()[['home', 'cabin', 'month']]
season_m_grouped = season_m.groupby('month').sum()
season_m_grouped.index = pd.to_datetime(
    season_m_grouped.index, format="%m").strftime("%b")

a = season_m_grouped.plot_bokeh.line(
    title="Monthly Power Consumption",
    ylabel="Consumption [kWh]",
    xlabel="Month",
    show_figure=True,
)


There are distinctive summer and winter seasonality for the cabin and winter seasonality for the main house.

In [250]:
season_m_box = season_m.copy()
season_m_box['month_str'] = pd.to_datetime(season_m_box.index).strftime("%b")
season_m_box.head(2)


,home,cabin,month,month_str
time,,,,
2021-01-01 01:00:00,0.860,0.16,1,Jan
2021-01-01 02:00:00,0.737,0.12,1,Jan


In [264]:
a = box(season_m_box, x='month_str', y='home', sort_by='month',
        title="Monthly Home Consumption Distribution",
        xlabel="Month", ylabel='Home Consumption', hover_mean_format='0.00')

In [256]:
a = box(season_m_box, x='month_str', y='cabin', sort_by='month',
        title="Monthly Cabin Consumption Distribution",
        xlabel="Month", ylabel='Cabin Consumption', hover_mean_format='0.00')

#### Holidays

In [37]:
temp = hourly_eda.copy()
# sum aggregate from hourly data to daily
temp_1d = temp[['cabin', 'home']].resample('1D').sum()

temp_date = temp.resample('1D').first().drop(['cabin', 'home'], axis='columns')
hols = temp_1D.merge(temp_date, how='left', left_index=True, right_index=True)
hols.index.name = 'date'
hols.head(2)

,cabin,home,month,day,hour,day_of_week,is_weekend,is_winter,is_summer,is_dst,rate,description,is_holiday
date,,,,,,,,,,,,,
2021-01-01,3.667,25.949,1,1,1,4,False,True,True,False,night,Uusaasta,True
2021-01-02,3.741,10.719,1,2,0,5,True,True,True,False,night,normal,False


Inspect if some of the features are causing highs or lows in the data.

In [38]:
temp = hourly_eda.copy()
temp = temp[['home', 'cabin']]

In [39]:
calendar(df=hols[['home', 'cabin', 'description', 'is_holiday', 'is_weekend']], 
         ys=['home', 'cabin'],
         exclude_values_dict={'description': ['normal']},
         width=600, height=350, 
         title='Holidays on Consumption', 
         ylabel='Daily Consumption [kWh]')

In [40]:
calendar(df=hols[['home', 'cabin', 'description', 'is_holiday', 'is_weekend']], 
         ys=['home', 'cabin'],
         exclude_values_dict={'description': ['normal']},
         width=600, height=350, 
         title='Holidays on Consumption', 
         ylabel='Daily Consumption [kWh]')

### Cycles
Inspect cyclic behaviour via moving average plots.

In [41]:
cyclic_d = season_d.copy()[['home', 'cabin']]
cyclic_d = cyclic_d.rolling(window=30, center=True).mean()

cyclic_d = cyclic_d.fillna(method='bfill').fillna(method='ffill')

a = cyclic_d.plot_bokeh.line(
    title='30-Day Moving Averages',
    ylabel='Consumption [kWh]',
    xlabel='Date',
    show_figure=False)

dt_formatter = DatetimeTickFormatter(
        days=["%b"],
        months=["%b"],
        years=["%Y"])

a.xaxis.formatter = dt_formatter
show(a)

30-Day moving average plot is very simialr to monthly seasonality plot, showing similar patterns.

#### Choosing Lags
Serial dependence in a time series will often become apparent by looking at a lag plot. 
 
The most commonly used measure of serial dependence is known as <code>autocorrelation</code>, which is simply the correlation a time series has with one of its lags.

In [42]:
correlogram(season_d['home'], lags=182, plot_width=700, plot_height=400)

Lags **1** and **32** seem to be significant. Lets inspect their respective lag plots i.e. direct correlations with the main feature.

In [43]:
plot_lags(season_d['home'], lags=[1,2,22,32,178], width=250, height=250, ncols=3)

Other than lag 1, we do not see direct correlation between lags and the main feature.

In [44]:
correlogram(season_d['cabin'], lags=182, plot_width=700, plot_height=400)

Useful lags:
* **1st** : carries information/correlation of previous day
* **32** : carries information/correlation of monthly cycles

In [45]:
a = lag_plot(season_d['home'], season_d['cabin'], lag=1, show_figure=True,
             plot_width=400, plot_height=400)



### Correlations

In [46]:
# correlation between home and cabin consumption
regplot(hols, 'cabin', 'home', reg=True, 
        extra_hover_tooltips=[('date', '@date{%d %b}')],
        hover_formatters={f"@date": 'datetime'},
        plot_width=600, plot_height=400)

In [47]:
plot_lags(x=hols['cabin'],
          y=hols['home'],
          lags=[1, 32],
          ncols=2,
          width=350,
          height=350)

In [48]:
temp = hols.copy()
temp['cabin_lag1'] = temp.cabin.shift(1).fillna(0)
temp['cabin_lag32'] = temp.cabin.shift(32).fillna(0)

plots = []
for f in ['cabin_lag1', 'cabin_lag32']:
    plots.append(regplot(temp, x=f, y='home',show_figure=False))
    
show(gridplot(plots, ncols=2, width=350, height=350))

**cabin** lags are not good leading indicators for **home** consumption

## Summary Stats
Summary statistics for years 2020 and 2021

In [319]:
monthly.head(2)

day     night     total
year locale month                               
2020 home   Yearly  2598.340  2000.977  4599.317
            Apr      183.016   140.080   323.096

In [342]:
plots = []
for place in ['home', 'cabin']:
    monthly_temp = monthly.loc[:,place,:]['total'].unstack('year').drop('Yearly')
    monthly_temp['month_nbr'] = pd.to_datetime(monthly_temp.index, format="%b").month
    monthly_temp = monthly_temp.reset_index().set_index('month_nbr').sort_index()
    
    a = monthly_home.plot_bokeh.bar(
    x='month',
    xlabel='Month',
    ylabel='Consumption',
    title=f'Monthly {place.capitalize()} Consumption',
    alpha=0.6,
    show_figure=False)
    plots.append(a)
    
a = pandas_bokeh.plot_grid([[plots[0]], [plots[1]]], width=700, height=400)

## Observations:
**Feature Engineering:**
* is_winter : 4 months - Dec, Jan, Feb, Mar. Selected by cold weather rather than winter months per se.
* is_summer : Jun, Jul, Aug
* is_dst : Daylight Saving Time (Mar 28 3am - Oct 31 4am)
* rate : daily or nightly price rate
* description : type of day, if ordinary day == "normal"
* is_holiday : if national holiday that day or not

**Distributions**
* hourly cabin & home : heavily skewed, use <code>PowerTrasnformer</code> to standardize and unskew.
* daily cabin & home : home has less skewness,  use <code>PowerTrasnformer</code> to standardize and unskew.

**Seasonality**
* **home** - strong hourly seasonality, but it might not be useful for our analysis later on as we are more conceerned about daily statistics rather than hourly. Strong annual and monthly seasonality, can be captured with 4 fourier features with Annual frequency.
* **cabin** : Medium annual and monthly seasonality, can be captured with 4 fourier features with Annual frequency.

**Remove**
* remove last observation as it shows for year 2022.

**Cycles - Lag Features**
* **home** and **cabin** : Lags 1 and 32 seem to be significant. We could use Lag 32 for forecasting.